# Minimization of different potentials
- Harmonic Potential
- Lennard-Jones (LJ) Potential
- Coulomb Potential
- Ewald Sum

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

from potentials import potentials, gradients
from distances import vectors, distances
from ewald_numpy import Ewald

from optimize import descent

import time

In [ ]:
def init_x(N=3, dim=2, x_pert=0, x_ampl=1., q_ampl=1, alter=False):
    """Generate initial particle distribution and charges
        N**dim - # of particles
        dim - dimension, 2D or 3D
        x_pert - position perturbation, displaces particles
        x_fact - position factor, sizes the free space between particles
        q_ampl - amplification of charges
        alter - True: enable alternating charges; False: generate random charges
    
    Output:
        x: positions
        q: charges
    """
    import random
    # Positions
    distri = np.linspace(-N, N, N)
    if dim==2:
        XX, YY = np.meshgrid(distri, distri)
        x = np.array([XX.flatten(),YY.flatten()]).T
    elif dim==3:
        XX, YY, ZZ = np.meshgrid(distri, distri, distri)
        x = np.array([XX.flatten(),YY.flatten(),ZZ.flatten()]).T
    assert len(x) == N**dim
    x += np.random.uniform(low=-x_pert, high=x_pert, size=(N**dim,dim))
    x *= x_ampl
    # Charges
    if alter:
        q = np.array([(-1)**(i+j) for i in range(N) for j in range(N**(dim-1))])
    else:
        q = np.array([[-1,1][random.randrange(2)] for i in range(N**dim)])
    q *= q_ampl
    return x, q

def Rand_Coord( N, dim=3, boxsize=(0,1) ):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N   (int): number of particles
        dim (int): dimension
        
    Output:
        x (float): position vectors (N x dim)
    """
    #import random
    #x = np.random.rand(N, dim)
    x = np.random.uniform(low=boxsize[0], high=boxsize[1], size=(N, dim))
    return x

def Rand_Charge( N, neutral=False, q_ampl=1 ):
    """Creates a list of N particles with random positions and charges
    
    Arguments:
        N        (int): Number of particles
        neutral (Bool): True: neutralizes system, requires even N
        
    Output:
        x, q
        x (float): position vectors (N x dim)
        q   (int): charges (N)
    """
    import random
    if neutral:
        assert N % 2 == 0
        q = np.array([(-1)**(i) for i in range(N)])
    else:
        q = np.array([[-1,1][random.randrange(2)] for i in range(N)])
    return q * q_ampl


## Harmonic Potential

In [ ]:
def grad_H2D(coord, boxsize=(0,1), pbc=False, r0=0.5, k=1):
    return gradients.harmonic(coord, boxsize, pbc, r0, k)

#def grad_H2D2(coord, boxsize=(0,1), pbc=False, r0=2, k=k):
#    return grad_H(coord, boxsize=(0,1), pbc=False, r0=r0, k=k)

example = np.array([[0.9, 0.9], [0.3, 0.3]])
t0 = time.time()
x_H2D, steps_H2D = descent(example, grad_H2D, boxsize=(0,1), pbc=True, prec=1e-7, save_config=True)
t1 = time.time()
print(x_H2D[-1], ' \n # of steps:', steps_H2D, ' \n time elapsed', t1-t0)

In [ ]:
x_init = x_H2D[0]
positions = x_H2D

fig, ax = plt.subplots(figsize=(4, 4))
scat = ax.scatter(x_init[:,0], x_init[:,1])
def animate(i):
    index = 50*i
    data = positions[index]
    scat.set_offsets(data)
    return scat
#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=25, bitrate=1800)
anim = animation.FuncAnimation(fig, animate, interval=1, repeat=False)
#anim.save('LJ_Harmonic_Particles.mp4', writer=writer)

In [ ]:
config = x_H2D
HO = np.sum(potentials.harmonic(config, boxsize=(0,1)), axis=-1)
print()
plt.plot(HO[0:-1])

In [ ]:
def grad_H_3D(coord, boxsize=(0,1), pbc=False, r0=.5, k=1):
    return gradients.harmonic(coord, boxsize, pbc, r0, k)

example = Rand_Coord(2000)
t0 = time.time()
#x_H_3D = descent(example, grad_H_3D, boxsize=(0,1), pbc=True, prec=1e-6, a=1e-3)
x_H_3D, steps_H_3D = descent(example, grad_H_3D, boxsize=(0,1), pbc=True, prec=1e-6, a=1e-3, save_config=True)
t1 = time.time()
print(x_H_3D.shape, ' \n # of steps:', steps_H_3D, ' \n time elapsed', t1-t0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

#x_init = x_H_3D[-1]
#x_init = x_H_3D[0]
x_init = x_H_3D[int(len(x_H_3D)/2)]
positions = x_H_3D

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection='3d')
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], x_init[:,2])
def animate(i):
    index = 20*i
    data = positions[index]
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    return scat,

#anim = animation.FuncAnimation(fig, animate, interval=1)
anim = animation.FuncAnimation(fig, animate, interval=10)#, frames=1050, repeat=False)

## Harmonic & LJ Potential

In [ ]:
sigma=1
def grad_HL(coord, boxsize=(0,1), pbc=False, r0=1, k=1, eps=1, sig=sigma):
    vecs = vectors(coord, boxsize, pbc)
    return gradients.harmonic(coord, boxsize, pbc, r0, k) + gradients.LJ(vecs, eps=eps, sig=sig)
   

In [ ]:
#N = 2
#x_HL_2D_init = Rand_Coord(N, dim=2, boxsize=(-1,1))*np.sqrt(N)

x_HL_2D_init = np.array([[2.,-2], [-2,2]])
#x_HL_2D_init = np.array([[-4.,1], [-2,1]])

print(x_HL_2D_init)

In [ ]:
t0 = time.time()
x_HL_2D, steps_HL_2D = descent(x_HL_2D_init, grad_HL, pbc=False, a=3e-3, prec=1e-6, maxst=1e6, save_config=True)
t1 = time.time()
print(steps_HL_2D, '\n', 'time elapsed: ', t1-t0)

In [ ]:
x_init = x_HL_2D[0]
positions = x_HL_2D


fig, ax = plt.subplots(figsize=(5, 5))
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1])#, c=q)
circles = [plt.Circle(r, radius=0.561*sigma, fill=False) 
            for i,r in enumerate(x_init)]
for c in circles:
    plt.gca().add_patch(c)
axlim=(-2,2)
ax.set_xlim(axlim)
ax.set_ylim((-1,3,))
def animate(i):
    index = 1*i
    data = positions[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    return scat
anim = animation.FuncAnimation(fig, animate, interval=1)

In [ ]:
x_init = x_HL_2D[427]
positions = x_HL_2D[427]

fig, ax = plt.subplots(figsize=(5, 5))
colors = np.arange(len(x_init))
#scat = ax.scatter(x_init[:,0], x_init[:,1])#, c=q)
circles = [plt.Circle(r, radius=0.561*sigma, fill=False) 
            for i,r in enumerate(x_init)]
for c in circles:
    plt.gca().add_patch(c)
axlim=(0,2)
ax.set_xlim(axlim)
ax.set_ylim(axlim)
ax.scatter(x_HL_2D[427][0], x_HL_2D[427][1])

x427= x_HL_2D[427]
print(np.linalg.norm(x_HL_2D[427]-1,axis=-1))
HO = np.sum(potentials.harmonic(x427, boxsize=(0,1)), axis=-1)
LJ = np.sum(np.array([potentials.LJ(distances(vectors(x427)))]))
print(HO+LJ)
#
configmin = x_HL_2D[-1]
print(np.linalg.norm(x_HL_2D[-1]-1,axis=-1))
HO = np.sum(potentials.harmonic(configmin, boxsize=(0,1)), axis=-1)
LJ = np.sum(np.array([potentials.LJ(distances(vectors(configmin)))]))
print(HO+LJ)

### Potential energy

In [ ]:
config = x_HL_2D
HO = np.sum(potentials.harmonic(config, boxsize=(0,1)), axis=-1)
LJ = np.array([potentials.LJ(distances(vectors(config[i]))) for i in range(len(config))])
#
fig1, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8,4))
ax1.plot(HO)
#ax1.plot(HO[424:434])
ax1.set_title("HO")
ax2.plot(LJ)
#ax2.plot(LJ[424:434])
ax2.set_title("LJ")
fig.tight_layout()

In [ ]:
sigma=1
def grad_HL(coord, boxsize=(0,1), pbc=False, r0=0.5, k=.1, eps=1, sig=sigma):
    vecs = vectors(coord, boxsize, pbc)
    return gradients.harmonic(coord, boxsize, pbc, r0, k) + gradients.LJ(vecs, eps=eps, sig=sig)
   
N = 40
#x_HL_3D_init = Random_particles(N, dim=3)[0]*np.sqrt(N)
x_HL_3D_init = init_x(N=3, dim=3)[0]
print(x_HL_3D_init.shape)

In [ ]:
t0 = time.time()
x_HL_3D, steps_HL_3D = descent(x_HL_3D_init, grad_HL, pbc=False, a=2e-3, prec=1e-5, save_config=True)
t1 = time.time()
print(steps_HL_3D, '\n', 'time elapsed: ', t1-t0)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

x_init = x_HL_3D[0]
positions = x_HL_3D

fig = plt.figure(figsize=(9,9))
ax = fig.add_subplot(111, projection='3d')
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], x_init[:,2])
def animate(i):
    index = 20*i
    data = positions[index]
    scat._offsets3d=(data[:,0],data[:,1],data[:,2])
    return scat,

#anim = animation.FuncAnimation(fig, animate, interval=1)
anim = animation.FuncAnimation(fig, animate, interval=10)#, frames=1050, repeat=False)

## Harmonic, LJ & Coulomb Potential

In [ ]:
N = 20
x_HLC_2D_init = Rand_Coord(N=N, dim=2, boxsize=(-1,1)) * np.sqrt(N) 
if N < 9:  x_HLC_2D_init *= 3
q = Rand_Charge(N=N, neutral=True, q_ampl=3)

sigma=1

def grad_HLC(coord, boxsize=(0,1), pbc=False, r0=0, k=.2, eps=1, sig=sigma):
    vecs = vectors(coord, boxsize, pbc)
    return gradients.harmonic(coord, boxsize, pbc, r0, k) + gradients.LJ(vecs, eps, sig) - gradients.coulomb(vecs, q)
    #return  gradients.harmonic(coord, boxsize, pbc, r0, k) - gradients.coulomb(vecs, q)

print(x_HLC_2D_init.shape, q.shape)

In [ ]:
#print(x_HLC_2D_init, q)
print(q)

In [ ]:
t0 = time.time()
x_HLC_2D, steps_HLC_2D = descent(x_HLC_2D_init, grad_HLC, pbc=False, a=1e-4, prec=1e-5, maxst=1e6, save_config=True)
t1 = time.time()
print(steps_HLC_2D, '\n', 'time elapsed: ', t1-t0)

In [ ]:
x_init = x_HLC_2D[0]
positions = x_HLC_2D


fig, ax = plt.subplots(figsize=(7, 7))
colors = np.arange(len(x_init))
scat = ax.scatter(x_init[:,0], x_init[:,1], c=q)
circles = [plt.Circle(r, radius=0.561*sigma, fill=False) 
            for i,r in enumerate(x_init)]
for c in circles:
    plt.gca().add_patch(c)
ax.set_xlim(-5, 5)
ax.set_ylim(-5, 5)
def animate(i):
    index = 40*i
    data = positions[index]
    scat.set_offsets(data)
    for i, c in enumerate(circles):
        c.center = data[i]
    return scat
anim = animation.FuncAnimation(fig, animate, interval=1)

In [ ]:
config = x_HLC_2D
HO = np.sum(potentials.harmonic(config, boxsize=(0,1)), axis=-1)
LJ = np.array([potentials.LJ(distances(vectors(config[i]))) for i in range(len(config))])
LJ = np.sum(LJ, axis=-1)
Coul = np.array([potentials.coulomb(distances(vectors(config[i])), q) for i in range(len(config))])
#Coul = np.sum(Coul, axis=-1)
print(HO[-1], LJ[-1], Coul[-1])
#
fig1, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(8,4))
#ax1.plot(HO[424:434])
ax1.plot(HO)
ax1.set_title("HO")
#ax2.plot(LJ[424:434])
ax2.plot(LJ)
ax2.set_title("LJ")
#ax2.plot(LJ[424:434])
ax3.plot(Coul)
ax3.set_title("Coulomb")
fig.tight_layout()

## Ewald

In [ ]:
E = Ewald(charges=q, boxsize=(0,1), alpha=5, r_cutoff=5, n_max=2, k_cutoff=5)
#E.params = (5, 3, 4, 4) # appropriate Ewald parameters for NaCl
charges = np.asarray([-1, -1, -1, -1, 1, 1, 1, 1.])
#charges = np.asarray([-1, -1, -1, -1, 1, 1, 1, 1])
pos = np.asarray([[0.05,0,0], [.5,.5,0], [.5,0,.5], [0,.5,.5], 
                  [.5,.5,.5], [.5,0,0], [0,.5,0],[0,0,.5]]) # slightly displace one particle

def grad_E(coord, boxsize=(0,1), pbc=True):
    return -E.force(coord)

#descent(pos, grad_E, pbc=True)